In [ ]:
# @title Шаг 1: Настройка среды
#
# Запустите эту ячейку один раз в начале работы, чтобы
# подключить Google Drive и установить необходимые библиотеки.

from google.colab import drive

# Монтирование Google Drive
print("1/2. Подключение Google Drive...")
drive.mount('/content/drive')

print("2/2. Установка animesub и других зависимостей...")
# Установка библиотеки animesub (без torch и torchaudio, которые уже установлены)
!pip install animesub ffmpeg-python > /dev/null

print("\nВсе зависимости установлены. Переходите к следующему шагу.")

In [ ]:
# @title Шаг 2: Настройки и запуск генерации
#
# Укажите путь к вашему файлу и выберите параметры.
# После этого запустите ячейку для начала генерации субтитров.

import os
import sys
import logging

# @markdown ### Пути к файлам
# @markdown Путь к файлу на Google Drive (например, `/content/drive/MyDrive/my_videos/episode1.mp4`)
input_file_path = "/content/drive/MyDrive/grappler_baki/Episode_4.mp4" #@param {type:"string"}

# @markdown Путь к папке для сохранения субтитров (например, `/content/drive/MyDrive/my_videos/`)
output_dir_path = "/content/drive/MyDrive/subtitles/" #@param {type:"string"}

# @markdown ### Параметры
# @markdown Имя модели Whisper (tiny, base, small, medium, large, large-v2, large-v3, kotoba-whisper, "kotoba-faster, kotoba-whisper-v2.2")
model_name = "kotoba-whisper" #@param ["tiny", "base", "small", "medium", "large", "large-v2", "large-v3", "kotoba-faster", "kotoba-whisper", "kotoba-whisper-v2.2"]

# @markdown Устройство для вычислений (cuda или cpu)
device = "cuda" #@param ["cuda", "cpu"]

# --- Выполнение ---
logging.info("\n--- Запуск процесса создания субтитров ---")

# Проверяем, существует ли входной файл
if not os.path.exists(input_file_path):
    logging.error(f"Ошибка: Файл не найден по пути: {input_file_path}")
else:
    # Проверяем, существует ли папка для сохранения. Если нет, создаем её.
    if not os.path.exists(output_dir_path):
        os.makedirs(output_dir_path)
        logging.info(f"Папка для сохранения '{output_dir_path}' была создана.")

    # Определяем имя выходного файла
    base_name = os.path.splitext(os.path.basename(input_file_path))[0]
    output_srt_name = os.path.join(output_dir_path, f"{base_name}.srt")

    # Запуск твоей команды animesub
    !animesub "$input_file_path" -o "$output_srt_name" -m "$model_name" -d "$device"

    logging.info("\n--- Процесс завершен! ---")
    # Проверка результата
    if os.path.exists(output_srt_name):
        logging.info(f"Субтитры успешно сохранены в: {output_srt_name}")
    else:
        logging.error(f"Файл субтитров не создан: {output_srt_name}")


In [ ]:
# @title Шаг 3: Скачать субтитры
#
# Запустите эту ячейку, чтобы скачать сгенерированный SRT-файл.
from google.colab import files

# Убедитесь, что переменная output_srt_name доступна
# Если вы запустили предыдущую ячейку, она будет доступна
try:
    files.download(output_srt_name)
    print(f"\nФайл {output_srt_name} успешно скачан.")
except NameError:
    print("Ошибка: Переменная 'output_srt_name' не определена. Сначала запустите предыдущую ячейку.")
except FileNotFoundError:
    print(f"Ошибка: Файл {output_srt_name} не найден. Проверьте путь.")